In [ ]:
import sys
sys.path.append('..')

In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

In [ ]:
from utils import helper

In [ ]:
root = '../..'
data_path = os.path.join(root, 'Data')

In [ ]:
ref_record = base_records = helper.read_fasta(os.path.join(data_path, 'sequences_425.fasta'))[0]
gen_records = helper.read_fasta(os.path.join(data_path, 'esm3_gen.fasta'))

In [ ]:
seq_ids = []
for rec in tqdm(gen_records):
    seq_ids.append(helper.calculate_seq_identity(ref_record, rec))

In [ ]:
plt.hist(seq_ids)